In [2]:
!pip install tensorflow opencv-python sklearn matplotlib mediapipe


[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

In [4]:
!pip install pyttsx3


[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
import pyttsx3
engine = pyttsx3.init()
engine.say("jaffer is gay")
engine.runAndWait()

mp holistic for keypoints extraction

In [8]:
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

In [9]:
def mediapipe_det(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

In [10]:
def draw_stlyed_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION,
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1))
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2))
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2))
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2))

extracting keypoints

In [11]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(1404)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

creating folders for data

In [117]:
DATA_PATH = os.path.join('MP_Data2')
actions = np.array(['hello','you','how'])
no_sequences = 30
sequence_length = 30

In [81]:
for action in actions:
    for sequence in range(no_sequences):
        try:
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass
        

data collecting

In [82]:

cap = cv2.VideoCapture(0)
time.sleep(4)
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    for action in actions:
        for sequence in range(no_sequences):
            for frame_num in range(sequence_length):
                
                ret, frame = cap.read()

                image, results = mediapipe_det(frame, holistic)
                #print(results)

                
                if frame_num == 0:
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0),2,cv2.LINE_AA)
                    cv2.putText(image, f'Collecting frames for {action} video number: {sequence}', (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255),1,cv2.LINE_AA)
                    cv2.waitKey(1000)

                else:
                    cv2.putText(image, f'Collecting frames for {action} video number: {sequence}', (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255),1,cv2.LINE_AA)
                
                draw_stlyed_landmarks(image, results)
                
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)
                
                
                cv2.imshow("frame", image)
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break



KeyboardInterrupt: 

In [123]:
keypoints[100]

0.6107305884361267

In [83]:
    cap.release()
    cv2.destroyAllWindows()

In [58]:
!pip install -U scikit-learn scipy matplotlib

data preprocessing

In [14]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [118]:
label_map = {label:num for num, label in enumerate(actions)}

In [119]:
label_map

{'hello': 0, 'you': 1, 'how': 2}

In [120]:
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), f"{frame_num}.npy"))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [121]:
X = np.array(sequences)

In [20]:
X.shape

(90, 30, 1662)

In [122]:
y = to_categorical(labels).astype(int)

In [48]:
y.shape

(120, 4)

In [123]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

building and training LSTM neural network

In [105]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [106]:
log_dir = os.path.join("Logs")
tb_callback = TensorBoard(log_dir=log_dir)

In [124]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [108]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [109]:
model.fit(x_train, y_train, epochs=2000, callbacks=[tb_callback])

Epoch 1/2000
4/4 [==============================] - 9s 288ms/step - loss: 4.8195 - categorical_accuracy: 0.1491
Epoch 2/2000
4/4 [==============================] - 1s 215ms/step - loss: 57.9392 - categorical_accuracy: 0.2807
Epoch 3/2000
4/4 [==============================] - 1s 189ms/step - loss: 61.2598 - categorical_accuracy: 0.2807
Epoch 4/2000
4/4 [==============================] - 1s 215ms/step - loss: 9.6124 - categorical_accuracy: 0.2281
Epoch 5/2000
4/4 [==============================] - 1s 170ms/step - loss: 24.4856 - categorical_accuracy: 0.1930
Epoch 6/2000
4/4 [==============================] - 1s 172ms/step - loss: 12.8312 - categorical_accuracy: 0.3158
Epoch 7/2000
4/4 [==============================] - 1s 182ms/step - loss: 16.2380 - categorical_accuracy: 0.2632
Epoch 8/2000
4/4 [==============================] - 1s 180ms/step - loss: 10.8769 - categorical_accuracy: 0.2544
Epoch 9/2000
4/4 [==============================] - 1s 173ms/step - loss: 10.4693 - categorical_ac

4/4 [==============================] - 1s 172ms/step - loss: 1.2832 - categorical_accuracy: 0.3772
Epoch 74/2000
4/4 [==============================] - 1s 183ms/step - loss: 1.2414 - categorical_accuracy: 0.3772
Epoch 75/2000
4/4 [==============================] - 1s 174ms/step - loss: 1.1975 - categorical_accuracy: 0.5351
Epoch 76/2000
4/4 [==============================] - 1s 164ms/step - loss: 1.2051 - categorical_accuracy: 0.4912
Epoch 77/2000
4/4 [==============================] - 1s 176ms/step - loss: 1.1713 - categorical_accuracy: 0.5789
Epoch 78/2000
4/4 [==============================] - 1s 171ms/step - loss: 1.1720 - categorical_accuracy: 0.5263
Epoch 79/2000
4/4 [==============================] - 1s 169ms/step - loss: 1.1654 - categorical_accuracy: 0.5526
Epoch 80/2000
4/4 [==============================] - 1s 191ms/step - loss: 1.1295 - categorical_accuracy: 0.5614
Epoch 81/2000
4/4 [==============================] - 1s 173ms/step - loss: 1.1121 - categorical_accuracy: 0.56

KeyboardInterrupt: 

In [144]:
model.load_weights('finalact1.h5')

In [125]:
model.save('action.h5')

In [49]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 30, 64)            442112    
                                                                 
 lstm_4 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_5 (LSTM)               (None, 64)                49408     
                                                                 
 dense_3 (Dense)             (None, 64)                4160      
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 3)                 99        
                                                                 
Total params: 596,675
Trainable params: 596,675
Non-tr

In [145]:
res = model.predict(x_test)

1/1 [==============================] - 0s 41ms/step


In [150]:
actions[np.argmax(res[3])]

'how'

In [149]:
actions[np.argmax(y_test[3])]

'how'

evaluating using confusion matrix 

In [31]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [151]:
yhat = model.predict(x_test)

1/1 [==============================] - 0s 38ms/step


In [152]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [153]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[4, 0],
        [0, 1]],

       [[3, 0],
        [0, 2]],

       [[3, 0],
        [0, 2]]], dtype=int64)

In [154]:
accuracy_score(ytrue, yhat)

1.0

In [99]:
import matplotlib as mlt
from sklearn import metrics

In [40]:
def speak(s):
    engine.say(s)
    engine.runAndWait()

In [ ]:
cap = cv2.VideoCapture('result1.mp4')
ret, frame = cap.read()

image, results = mediapipe_det(frame, holistic)        
draw_stlyed_landmarks(image, results)
cv2.sam

real time detection 

In [181]:
sequence = []
sentence = []
predictions = []
threshold = 0.9
qq=0

cap = cv2.VideoCapture('result1.mp4')
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()

        image, results = mediapipe_det(frame, holistic)
        #print(results)
        
        draw_stlyed_landmarks(image, results)
        
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
            
            if np.unique(predictions[-10:])[0]==np.argmax(res):
                if res[np.argmax(res)] > threshold:
                    
                    if len(sentence) > 0:
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])
            if len(sentence) > 5:
                sentence = sentence[-5:]
        
        cv2.rectangle(image, (0,0), (640,40), (245,117,16), -1)
        
        cv2.imshow("frame", image)
        
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()


1/1 [==============================] - 0s 22ms/step
hello
1/1 [==============================] - 0s 16ms/step
hello
1/1 [==============================] - 0s 31ms/step
hello
1/1 [==============================] - 0s 31ms/step
hello
1/1 [==============================] - 0s 28ms/step
hello
1/1 [==============================] - 0s 16ms/step
hello
1/1 [==============================] - 0s 16ms/step
hello
1/1 [==============================] - 0s 31ms/step
hello
1/1 [==============================] - 0s 22ms/step
hello
1/1 [==============================] - 0s 31ms/step
hello
1/1 [==============================] - 0s 31ms/step
hello
1/1 [==============================] - 0s 22ms/step
hello
1/1 [==============================] - 0s 31ms/step
hello
1/1 [==============================] - 0s 16ms/step
hello
1/1 [==============================] - 0s 16ms/step
hello
1/1 [==============================] - 0s 16ms/step
hello
1/1 [==============================] - 0s 7ms/step
hello
1/1 [==========

In [179]:
    cap.release()
    cv2.destroyAllWindows()

In [54]:
speak(pred_sent(sentence))